In [1]:
import numpy as np                                                    # Packages for data analysis
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch, Circle
from scipy.ndimage import gaussian_filter1d
import braingeneers                                                   # Braingeneers code
from braingeneers.analysis.analysis import SpikeData, read_phy_files
from ipywidgets import interact, interactive, fixed, interact_manual  # package for interactive widgets 
import ipywidgets as ipw
from IPython.display import HTML, display, Javascript, clear_output
from scipy.sparse import csr_matrix
import scipy.io
from scipy.sparse import coo_matrix
import numpy as np
from decimal import Decimal, getcontext

In [2]:
sd = read_phy_files('/home/jovyan/work/Experiment/data/ephys/2023-04-02-hc328_rec/derived/kilosort2/2023_04_02_hc328_5_curated.zip')

/tmp/ipykernel_22640/4253654759.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) read_phy_files. (Prefer load_spike_data()) -- Deprecated since version 0.1.13.
  sd = read_phy_files('/home/jovyan/work/Experiment/data/ephys/2023-04-02-hc328_rec/derived/kilosort2/2023_04_02_hc328_5_curated.zip')


In [3]:
s_train = sd.train
neural_unit_index = []
spike_times = []
for index, array in enumerate(s_train):
    neural_unit_index.extend([index] * len(array))
    spike_times.extend(array)
    
result_array = np.column_stack((neural_unit_index, spike_times))

result_array[:,1] /= 1000
result_array[:, 0] += 1

In [4]:
dataset_5 = scipy.io.loadmat('matlab_human_5_FCM.mat')

In [5]:
dataset_5

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Tue Aug  1 23:26:40 2023',
 '__version__': '1.0',
 '__globals__': [],
 'FC_MAT': array([[0.00000000e+00, 0.00000000e+00, 2.76851730e-03, ...,
         0.00000000e+00, 0.00000000e+00, 4.57084177e-04],
        [4.63933372e-04, 0.00000000e+00, 7.14599232e-04, ...,
         0.00000000e+00, 5.78875085e-03, 0.00000000e+00],
        [3.97916140e-03, 0.00000000e+00, 0.00000000e+00, ...,
         5.19823297e-04, 1.02774970e-02, 3.64786849e-04],
        ...,
        [0.00000000e+00, 1.30224552e-03, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.96898976e-04, 0.00000000e+00, 0.00000000e+00, ...,
         1.18719219e-03, 0.00000000e+00, 0.00000000e+00],
        [1.38286235e-05, 0.00000000e+00, 7.27556377e-04, ...,
         0.00000000e+00, 1.92851744e-03, 0.00000000e+00]])}

In [ ]:
def createFCM(spksExc, spksInh=None):    
    
    # time constant of exponential profile (used for coupling)
    tau = 3e-3

    # 1e-4 binless window step in seconds
    step = 1e-4
    
    # store ids of neurons that really fired -- corresponding to entries in
    # adjacency relation
    
    if spksInh is None:
        old_ids = np.unique(spksExc[:, 0])
        spks = np.column_stack((spksExc, np.ones(len(spksExc))))
    else:
        old_ids = np.unique(np.concatenate((spksExc[:, 0], spksInh[:, 0])))
        spks = np.row_stack((np.column_stack((spksExc, np.ones(len(spksExc)))), 
                             np.column_stack((spksInh, -np.ones(len(spksInh))))))
        
    Eneuron_end = np.max(spksExc[:, 0])
    num_neurons = len(old_ids)
    
    row_indices = old_ids - 1  # Since Python uses 0-based indexing, we subtract 1 from old_ids
    col_indices = np.zeros_like(row_indices)
    values = np.arange(1, num_neurons + 1)

    new_ids = coo_matrix((values, (row_indices, col_indices)), shape=(num_neurons, 1))
    
    # frequency of spiking for individual neurons
    spike_freq = np.zeros(num_neurons)
    
    # duration of spike trains
    t_min = 0
    t_max = np.max(spksExc[:,1])
    T = t_max - t_min
    sig_length = np.ceil(T/step).astype(int)
    
    # Convert the new_ids sparse matrix to a dense NumPy array
    new_ids_array = new_ids.toarray().flatten()

    print(Eneuron_end)

    # Get spiking frequencies
    spike_freq = np.zeros(num_neurons)

    for i in range(1, num_neurons + 1):
        if i <= Eneuron_end:
            neuron_spikes = spksExc[spksExc[:, 0] == old_ids[i - 1], 1]
            spike_freq[i - 1] = np.sum(neuron_spikes) / T
        else:
            if spksInh is None:
                print("spksInh is None. Make sure it is properly defined.")
                return None
                
            neuron_spikes = spksInh[spksInh[:, 0] == old_ids[i - 1], 1]
            spike_freq[i - 1] = np.sum(neuron_spikes) / T
            
    # create signals for each neuron        
    signals = np.zeros((num_neurons, sig_length))
    for i in range(1, num_neurons + 1):
        k = old_ids[i - 1]
        IorE = np.unique(spks[spks[:, 0] == k, 2])
        # how neuron i will influence other neurons
        signals[i - 1, :] = create_signal(spks[spks[:, 0] == k, 1], step, sig_length, tau, IorE)
    
    FC_MAT = np.zeros((num_neurons, num_neurons))
    spike_vecs = np.zeros((num_neurons, sig_length))

    # Previous code up to combining with discrete spiking events...

    # Combine with discrete spiking events
    for i in range(1, num_neurons + 1):
        k = old_ids[i - 1]
        spike_times = spks[spks[:, 0] == k, 1]
        spike_times_vec = coo_matrix((np.ones_like(spike_times), 
                                      (np.ceil(spike_times / step).astype(int) - 1, 
                                       np.zeros_like(spike_times))), 
                                     shape=(sig_length, 1)).toarray().flatten()
        spike_vecs[i - 1, :] = spike_times_vec
        # Influence on neuron i from all other neurons.
        FC_MAT[:, i - 1] = signals.dot(spike_times_vec)

    # Thresholding
    FC_MAT[FC_MAT < 0] = 0
    FC_MAT = FC_MAT - np.diag(np.diag(FC_MAT))

    # Frequency normalization
    freq_normalization = spike_freq / np.maximum(spike_freq, 1)  # Divide element-wise with maximum value of 1
    freq_normalization[freq_normalization > 1] = 1

    FC_MAT = FC_MAT * freq_normalization

    return FC_MAT

In [ ]:
def create_signal(start_times, dt, sig_length, tau, mode):
    # time vector
    t_vec = np.arange(dt, (sig_length + 1) * dt, dt)

    q = np.zeros_like(t_vec)

    if mode == 1:
        for k in range(len(start_times)):
            # t_vec - start_times[k]
            
            qq = np.exp(-(t_vec - start_times[k]) / tau)
            qq[qq > 1] = 0
            if k < len(start_times) - 1:
                qq[t_vec >= start_times[k + 1]] = 0
            q = q + qq
    else:
        for k in range(len(start_times)):
            qq = np.exp(-(t_vec - start_times[k]) / tau)
            qq[qq > 1] = 0
            qq = 1 - qq
            qq[qq > 0.99] = 0  # corresponds to ~4.5 tau
            if k < len(start_times) - 1:
                qq[t_vec >= start_times[k + 1]] = 0
            q = q + qq

    # due to rounding of spiking times, the exponential may sometimes not start at 1
    if mode == 1:
        idx = np.ceil(start_times / dt).astype(int) - 1
        idx = np.clip(idx, 0, len(q) - 1)  # Clip indices to be within valid range
        q[idx] = 1

    # shift to zero mean and make magnitude normalized
    q = q - np.mean(q)
    q = q / (np.max(q) * len(start_times))

    return q

In [ ]:
FC_MAT = createFCM(result_array)

In [ ]:
python_data = FC_MAT

In [ ]:
matlab_data = dataset_5['FC_MAT']

In [ ]:
python_data

In [ ]:
matlab_data

In [ ]:
diff_matrix = matlab_data - python_data

In [ ]:
diff_matrix

In [ ]:
data_1 = matlab_data
data_2 = python_data

# Calculate the common vmin and vmax for both datasets
vmin = min(data_1.min(), data_2.min())
vmax = max(data_1.max(), data_2.max())

# Create a 1x2 subplot to place the heatmaps side by side
fig, axs = plt.subplots(1, 2, figsize=(15, 8))  # Adjust the figure size as needed

# Plot the first heatmap (dataset_5)
heatmap1 = axs[0].imshow(data_1, cmap='viridis', interpolation='nearest', aspect='equal', vmin=vmin, vmax=vmax)
axs[0].set_title('Heatmap of dataset_5')

# Plot the second heatmap (python_data)
heatmap2 = axs[1].imshow(data_2, cmap='viridis', interpolation='nearest', aspect='equal', vmin=vmin, vmax=vmax)
axs[1].set_title('Heatmap of python_data')

# Add colorbars for better interpretation
colorbar1 = plt.colorbar(heatmap1, ax=axs[0], fraction=0.03, pad=0.04)
colorbar1.set_label('Strength of connectivity')

colorbar2 = plt.colorbar(heatmap2, ax=axs[1], fraction=0.03, pad=0.04)
colorbar2.set_label('Strength of connectivity')

plt.show()

In [ ]:
import numpy as np

data_1 = matlab_data
data_2 = python_data

# Calculate the common vmin and vmax for both datasets
vmin = min(data_1.min(), data_2.min())
vmax = max(data_1.max(), data_2.max())

# Create a 1x3 subplot to place the heatmaps side by side
fig, axs = plt.subplots(1, 3, figsize=(20, 8))  # Adjust the figure size as needed

# Plot the first heatmap (dataset_5)
heatmap1 = axs[0].imshow(data_1, cmap='viridis', interpolation='nearest', aspect='equal', vmin=vmin, vmax=vmax)
axs[0].set_title('MATLAB FCM')

# Plot the second heatmap (python_data)
heatmap2 = axs[1].imshow(data_2, cmap='viridis', interpolation='nearest', aspect='equal', vmin=vmin, vmax=vmax)
axs[1].set_title('Python FCM')

# Calculate the difference matrix
diff_matrix = data_1 - data_2

diff_vmax = np.abs(diff_matrix).max()
heatmap3 = axs[2].imshow(diff_matrix, cmap='coolwarm', interpolation='nearest', aspect='equal', vmin=-diff_vmax, vmax=diff_vmax)
axs[2].set_title('Difference between MATLAB and Python FCM')

# Add colorbars for better interpretation
colorbar1 = plt.colorbar(heatmap1, ax=axs[0], fraction=0.03, pad=0.04)
colorbar1.set_label('Strength of connectivity')

colorbar2 = plt.colorbar(heatmap2, ax=axs[1], fraction=0.03, pad=0.04)
colorbar2.set_label('Strength of connectivity')

colorbar3 = plt.colorbar(heatmap3, ax=axs[2], fraction=0.03, pad=0.04)
colorbar3.set_label('Difference')

plt.savefig('python_matlab_difference.png')

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

def correlation(sd):
    # Correlation
    corr = np.zeros((sd.N, sd.N)) #inds by inds

    dense_raster = sd.raster(bin_size=1) # in ms
    sigma = 5                            # Blur it
    dense_raster = gaussian_filter1d(dense_raster.astype(float), sigma=sigma) 
    corr = np.corrcoef(dense_raster)
    return corr

# Assume you have already defined sd and FC_MAT

# Get the correlation matrix from the first code
corr_matrix = correlation(sd)

# Create a 2x2 subplot arrangement
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# Plot the correlation matrix on the first subplot
heatmap1 = axs[0].imshow(corr_matrix, cmap='viridis')
axs[0].set_title('Correlation Matrix (2023_04_02_hc328_5_curated)')
axs[0].set_xlabel("Neuron index")
axs[0].set_ylabel("Neuron index")

# Plot the functional connectivity matrix on the second subplot
heatmap2 = axs[1].imshow(matlab_data, cmap='viridis', aspect='equal')
axs[1].set_title('Functional Connectivity Matrix (2023_04_02_hc328_5_curated)')
axs[1].set_xlabel('Neuron Index')
axs[1].set_ylabel('Neuron Index')

# Overlay the correlation and functional connectivity matrices on the third subplot
overlay = corr_matrix * matlab_data
overlay_min = np.min(overlay)
overlay_max = np.percentile(overlay, 99.92)  # Adjust percentile as needed
heatmap3 = axs[2].imshow(overlay, cmap='coolwarm', aspect='equal', vmin=overlay_min, vmax=overlay_max)
axs[2].set_title('Correlation FCM Interaction')
axs[2].set_xlabel('Neuron Index')
axs[2].set_ylabel('Neuron Index')

# Add colorbars for better interpretation
colorbar1 = plt.colorbar(heatmap1, ax=axs[0], fraction=0.03, pad=0.04)
colorbar1.set_label('Correlation Coefficient')

colorbar2 = plt.colorbar(heatmap2, ax=axs[1], fraction=0.03, pad=0.04)
colorbar2.set_label('Strength of connectivity')

colorbar3 = plt.colorbar(heatmap3, ax=axs[2], fraction=0.03, pad=0.04)
colorbar3.set_label('Interaction value')

plt.tight_layout()

plt.savefig('correlation_fcm_interaction.png')

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

def correlation(sd):
    # Correlation
    corr = np.zeros((sd.N, sd.N)) #inds by inds

    dense_raster = sd.raster(bin_size=1) # in ms
    sigma = 5                            # Blur it
    dense_raster = gaussian_filter1d(dense_raster.astype(float), sigma=sigma) 
    corr = np.corrcoef(dense_raster)
    return corr

def plot_sttc(sd):
    sttc = np.zeros((sd.N, sd.N))
    for i in range(sd.N):
        for j in range(sd.N):
            sttc[i][j] = sd.spike_time_tiling(i,j)
    return sttc
#     plt.imshow(sttc)
#     plt.xlabel("Neuron index")
#     plt.ylabel("Neuron index")
#     plt.show()

# Assume you have already defined sd and FC_MAT

# Get the correlation matrix from the first code
# corr_matrix = correlation(sd)
sttc_matrix = plot_sttc(sd)

# Create a 2x2 subplot arrangement
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# Plot the correlation matrix on the first subplot
heatmap1 = axs[0].imshow(sttc_matrix, cmap='viridis')
axs[0].set_title('STTC (2023_04_02_hc328_5_curated)')
axs[0].set_xlabel("Neuron index")
axs[0].set_ylabel("Neuron index")

# Plot the functional connectivity matrix on the second subplot
heatmap2 = axs[1].imshow(matlab_data, cmap='viridis', aspect='equal')
axs[1].set_title('Functional Connectivity Matrix (2023_04_02_hc328_5_curated)')
axs[1].set_xlabel('Neuron Index')
axs[1].set_ylabel('Neuron Index')

# Overlay the correlation and functional connectivity matrices on the third subplot
overlay = sttc_matrix * matlab_data
overlay_min = np.min(overlay)
overlay_max = np.percentile(overlay, 99.92)  # Adjust percentile as needed
heatmap3 = axs[2].imshow(overlay, cmap='coolwarm', aspect='equal', vmin=overlay_min, vmax=overlay_max)
axs[2].set_title('STTC FCM Interaction')
axs[2].set_xlabel('Neuron Index')
axs[2].set_ylabel('Neuron Index')

# Add colorbars for better interpretation
colorbar1 = plt.colorbar(heatmap1, ax=axs[0], fraction=0.03, pad=0.04)
colorbar1.set_label('STTC')

colorbar2 = plt.colorbar(heatmap2, ax=axs[1], fraction=0.03, pad=0.04)
colorbar2.set_label('Strength of connectivity')

colorbar3 = plt.colorbar(heatmap3, ax=axs[2], fraction=0.03, pad=0.04)
colorbar3.set_label('Interaction value')

plt.tight_layout()

plt.savefig('sttc_fcm_interaction.png')

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

def correlation(sd):
    # Correlation
    corr = np.zeros((sd.N, sd.N)) #inds by inds

    dense_raster = sd.raster(bin_size=1) # in ms
    sigma = 5                            # Blur it
    dense_raster = gaussian_filter1d(dense_raster.astype(float), sigma=sigma) 
    corr = np.corrcoef(dense_raster)
    return corr

def plot_sttc(sd):
    sttc = np.zeros((sd.N, sd.N))
    for i in range(sd.N):
        for j in range(sd.N):
            sttc[i][j] = sd.spike_time_tiling(i,j)
    return sttc

# ... (assuming sd and matlab_data are defined) ...

# Get the correlation matrix from the first code
# corr_matrix = correlation(sd)
sttc_matrix = plot_sttc(sd)

# Create a 2x2 subplot arrangement
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# Plot the correlation matrix on the first subplot
heatmap1 = axs[0].imshow(sttc_matrix, cmap='viridis')
axs[0].set_title('STTC (2023_04_02_hc328_5_curated)')
axs[0].set_xlabel("Neuron index")
axs[0].set_ylabel("Neuron index")

# Plot the functional connectivity matrix on the second subplot
heatmap2 = axs[1].imshow(matlab_data, cmap='viridis', aspect='equal')
axs[1].set_title('Functional Connectivity Matrix (2023_04_02_hc328_5_curated)')
axs[1].set_xlabel('Neuron Index')
axs[1].set_ylabel('Neuron Index')

# Overlay the correlation and functional connectivity matrices on the third subplot
overlay = sttc_matrix * matlab_data
overlay_min = np.min(overlay)
overlay_max = np.percentile(overlay, 99.92)  # Adjust percentile as needed
heatmap3 = axs[2].imshow(overlay, cmap='coolwarm', aspect='equal', vmin=overlay_min, vmax=overlay_max)
axs[2].set_title('STTC FCM Interaction')
axs[2].set_xlabel('Neuron Index')
axs[2].set_ylabel('Neuron Index')

# Find the indices of the maximum value in the overlay matrix
max_idx = np.unravel_index(np.argmax(overlay), overlay.shape)
max_row, max_col = max_idx

# Add a text annotation with the highest interaction value and its indices
highest_interaction = overlay[max_row, max_col]
annotation_text = f'Highest Interaction: {highest_interaction:.2f}\nNeuron Index: ({max_row}, {max_col})'
axs[2].text(0.05, 0.95, annotation_text, transform=axs[2].transAxes, fontsize=12,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

# Add colorbars for better interpretation
colorbar1 = plt.colorbar(heatmap1, ax=axs[0], fraction=0.03, pad=0.04)
colorbar1.set_label('STTC')

colorbar2 = plt.colorbar(heatmap2, ax=axs[1], fraction=0.03, pad=0.04)
colorbar2.set_label('Strength of connectivity')

colorbar3 = plt.colorbar(heatmap3, ax=axs[2], fraction=0.03, pad=0.04)
colorbar3.set_label('Interaction value')

plt.tight_layout()

plt.savefig('sttc_fcm_interaction.png')

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

def correlation(sd):
    # Correlation
    corr = np.zeros((sd.N, sd.N)) #inds by inds

    dense_raster = sd.raster(bin_size=1) # in ms
    sigma = 5                            # Blur it
    dense_raster = gaussian_filter1d(dense_raster.astype(float), sigma=sigma) 
    corr = np.corrcoef(dense_raster)
    return corr

def plot_sttc(sd):
    sttc = np.zeros((sd.N, sd.N))
    for i in range(sd.N):
        for j in range(sd.N):
            sttc[i][j] = sd.spike_time_tiling(i,j)
    return sttc

# ... (assuming sd and matlab_data are defined) ...

# Get the correlation matrix from the first code
# corr_matrix = correlation(sd)
sttc_matrix = plot_sttc(sd)

# Overlay the correlation and functional connectivity matrices on the third subplot
overlay = sttc_matrix * matlab_data
overlay_min = np.min(overlay)
overlay_max = np.percentile(overlay, 99.92)  # Adjust percentile as needed

# Find the indices of the top 5 interactions in the overlay matrix in reverse order
top5_indices = np.unravel_index(np.argsort(overlay.ravel())[-1:-6:-1], overlay.shape)
top5_rows, top5_cols = top5_indices

# Create a 1x1 subplot arrangement
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the overlay with colormap and circles around the top 5 interactions
heatmap = ax.imshow(overlay, cmap='coolwarm', aspect='equal', vmin=overlay_min, vmax=overlay_max)

# Add circles around the top 5 interactions with bigger circle size
circle_size = 1.5
for row, col in zip(top5_rows, top5_cols):
    circle = plt.Circle((col, row), circle_size, color='red', linewidth=2, fill=False)
    ax.add_artist(circle)

# Add a text annotation for each of the top 5 interactions to the left of the heatmap
for i, (row, col) in enumerate(zip(top5_rows, top5_cols)):
    interaction_value = overlay[row, col]
    neuron_indices = f'({row}, {col})'
    annotation_text = f'Top {i + 1}: {interaction_value:.2f}\nNeuron Index: {neuron_indices}'
    ax.text(-0.12, 0.95 - i * 0.11, annotation_text, transform=ax.transAxes, fontsize=12,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

ax.set_title('STTC FCM Interaction')
ax.set_xlabel('Neuron Index')
ax.set_ylabel('Neuron Index')

# Add a colorbar for better interpretation to the right of the heatmap
colorbar = plt.colorbar(heatmap, ax=ax, fraction=0.03, pad=0.07)
colorbar.set_label('Interaction value')

plt.tight_layout()

# plt.savefig('top5_sttc_fcm_interaction.png')

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from matplotlib.patches import ConnectionPatch

def correlation(sd):
    # Correlation
    corr = np.zeros((sd.N, sd.N)) #inds by inds

    dense_raster = sd.raster(bin_size=1) # in ms
    sigma = 5                            # Blur it
    dense_raster = gaussian_filter1d(dense_raster.astype(float), sigma=sigma) 
    corr = np.corrcoef(dense_raster)
    return corr

def plot_sttc(sd):
    sttc = np.zeros((sd.N, sd.N))
    for i in range(sd.N):
        for j in range(sd.N):
            sttc[i][j] = sd.spike_time_tiling(i,j)
    return sttc

# ... (assuming sd and matlab_data are defined) ...

# Get the correlation matrix from the first code
# corr_matrix = correlation(sd)
sttc_matrix = plot_sttc(sd)

# Overlay the correlation and functional connectivity matrices on the third subplot
overlay = sttc_matrix * matlab_data
overlay_min = np.min(overlay)
overlay_max = np.percentile(overlay, 99.92)  # Adjust percentile as needed

# Find the indices of the top 5 interactions in the overlay matrix in reverse order
top5_indices = np.unravel_index(np.argsort(overlay.ravel())[-1:-6:-1], overlay.shape)
top5_rows, top5_cols = top5_indices

# Create a 1x1 subplot arrangement
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the overlay with colormap and circles around the top 5 interactions
heatmap = ax.imshow(overlay, cmap='coolwarm', aspect='equal', vmin=overlay_min, vmax=overlay_max)

# Add circles around the top 5 interactions with bigger circle size
circle_size = 1.5
for row, col in zip(top5_rows, top5_cols):
    circle = plt.Circle((col, row), circle_size, color='red', linewidth=2, fill=False)
    ax.add_artist(circle)

# Add a text annotation for each of the top 5 interactions and connecting arrows
for i, (row, col) in enumerate(zip(top5_rows, top5_cols)):
    interaction_value = overlay[row, col]
    neuron_indices = f'({row}, {col})'
    annotation_text = f'Top {i + 1}: {interaction_value:.2f}\nNeuron Index: {neuron_indices}'
    ax.text(col + 0.7, row, annotation_text, fontsize=12, verticalalignment='center',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

    # Add connecting arrows from label to circle
    con = ConnectionPatch((col + 0.5, row), (col, row), "data", "data",
                          arrowstyle='->', lw=2, color='black')
    ax.add_artist(con)

ax.set_title('STTC FCM Interaction')
ax.set_xlabel('Neuron Index')
ax.set_ylabel('Neuron Index')

# Remove the colorbar
cax = fig.add_axes([0.0, 0.0, 0.0, 0.0])  # Create an empty axis for the colorbar
colorbar = plt.colorbar(heatmap, cax=cax)

plt.tight_layout()

plt.savefig('top5_sttc_fcm_interaction.png')

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from matplotlib.patches import ConnectionPatch

def correlation(sd):
    # Correlation
    corr = np.zeros((sd.N, sd.N)) #inds by inds

    dense_raster = sd.raster(bin_size=1) # in ms
    sigma = 5                            # Blur it
    dense_raster = gaussian_filter1d(dense_raster.astype(float), sigma=sigma) 
    corr = np.corrcoef(dense_raster)
    return corr

def plot_sttc(sd):
    sttc = np.zeros((sd.N, sd.N))
    for i in range(sd.N):
        for j in range(sd.N):
            sttc[i][j] = sd.spike_time_tiling(i,j)
    return sttc

# ... (assuming sd and matlab_data are defined) ...

# Get the correlation matrix from the first code
# corr_matrix = correlation(sd)
sttc_matrix = plot_sttc(sd)

# Overlay the correlation and functional connectivity matrices on the third subplot
overlay = sttc_matrix * matlab_data
overlay_min = np.min(overlay)
overlay_max = np.percentile(overlay, 99.92)  # Adjust percentile as needed

# Find the indices of the top 5 interactions in the overlay matrix in reverse order
top5_indices = np.unravel_index(np.argsort(overlay.ravel())[-1:-6:-1], overlay.shape)
top5_rows, top5_cols = top5_indices

# Create a 1x1 subplot arrangement
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the overlay with colormap and circles around the top 5 interactions
heatmap = ax.imshow(overlay, cmap='coolwarm', aspect='equal', vmin=overlay_min, vmax=overlay_max)

# Add circles around the top 5 interactions with bigger circle size
circle_size = 3.0
for row, col in zip(top5_rows, top5_cols):
    circle = plt.Circle((col, row), circle_size, color='red', linewidth=2, fill=False)
    ax.add_artist(circle)

# Add a text annotation for each of the top 5 interactions and connecting arrows
label_offset = 4.0
for i, (row, col) in enumerate(zip(top5_rows, top5_cols)):
    interaction_value = overlay[row, col]
    neuron_indices = f'({row}, {col})'
    annotation_text = f'Top {i + 1}: {interaction_value:.2f}\nNeuron Index: {neuron_indices}'
    
    # Find the best available position for the label to avoid overlap
    best_position = False
    text_y = row
    while not best_position:
        text_x = col + label_offset
        best_position = True
        for text in ax.texts:
            existing_bbox = text.get_window_extent(renderer=fig.canvas.get_renderer())
            new_bbox = ax.text(text_x, text_y, annotation_text, fontsize=12, 
                               verticalalignment='center').get_window_extent(renderer=fig.canvas.get_renderer())
            if existing_bbox.overlaps(new_bbox):
                best_position = False
                text_y += 1.0
                break
    
    # Add the label to the plot
    ax.text(text_x, text_y, annotation_text, fontsize=12, verticalalignment='center',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
    
    # Add connecting arrows from label to circle
    con = ConnectionPatch((col + circle_size, row), (col, row), "data", "data",
                          arrowstyle='->', lw=2, color='black')
    ax.add_artist(con)

ax.set_title('STTC FCM Interaction')
ax.set_xlabel('Neuron Index')
ax.set_ylabel('Neuron Index')

plt.tight_layout(rect=[0, 0, 0.85, 1])  # Adjust the rect parameter to create space for the labels

# Remove the colorbar
cax = fig.add_axes([0.87, 0.15, 0.02, 0.7])  # Create an empty axis for the colorbar
colorbar = plt.colorbar(heatmap, cax=cax)

plt.savefig('top5_sttc_fcm_interaction.png')

plt.show()

In [ ]:
print("Hi")